In [1]:
# ! pip install transformers datasets
# ! pip install transformers datasets evaluate sacrebleu

In [2]:
import evaluate
import numpy as np
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline

In [3]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "
metric = evaluate.load("sacrebleu")

In [4]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [5]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [6]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [7]:
books = load_dataset("opus_books", "en-fr")
books = books["train"].train_test_split(test_size=0.2)
print(books["train"][0])

tokenized_books = books.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Found cached dataset opus_books (C:/Users/cpb5867/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '107519', 'translation': {'en': 'Chance alone, it was clear, had brought Colonel Proctor on this train; but there he was, and it was necessary, at all hazards, that Phileas Fogg should not perceive his adversary.', 'fr': "Évidemment, c'était le hasard seul qui avait amené dans ce train le colonel Proctor, mais enfin il y était, et il fallait empêcher à tout prix que Phileas Fogg aperçut son adversaire."}}


Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

Cloning https://huggingface.co/cpb5867/my_awesome_opus_books_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/231M [00:00<?, ?B/s]

Download file training_args.bin:  11%|#         | 412/3.68k [00:00<?, ?B/s]

Clean file training_args.bin:  27%|##7       | 1.00k/3.68k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

In [8]:
trainer.train()
trainer.push_to_hub()

c:\Users\cpb5867\.conda\envs\TransConsf\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12710 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.1063, 'learning_rate': 1.9213217938631003e-05, 'epoch': 0.08}
{'loss': 2.0124, 'learning_rate': 1.8426435877261997e-05, 'epoch': 0.16}
{'loss': 1.9648, 'learning_rate': 1.7639653815893e-05, 'epoch': 0.24}
{'loss': 1.9318, 'learning_rate': 1.6852871754524e-05, 'epoch': 0.31}
{'loss': 1.9092, 'learning_rate': 1.6067663257277734e-05, 'epoch': 0.39}
{'loss': 1.9191, 'learning_rate': 1.5280881195908735e-05, 'epoch': 0.47}
{'loss': 1.8919, 'learning_rate': 1.4494099134539735e-05, 'epoch': 0.55}
{'loss': 1.8842, 'learning_rate': 1.3707317073170734e-05, 'epoch': 0.63}
{'loss': 1.8823, 'learning_rate': 1.292210857592447e-05, 'epoch': 0.71}
{'loss': 1.8579, 'learning_rate': 1.213532651455547e-05, 'epoch': 0.79}
{'loss': 1.8752, 'learning_rate': 1.1348544453186468e-05, 'epoch': 0.87}
{'loss': 1.8511, 'learning_rate': 1.0561762391817467e-05, 'epoch': 0.94}


  0%|          | 0/1589 [00:00<?, ?it/s]

...

  0%|          | 0/1589 [00:00<?, ?it/s]

{'eval_loss': 1.6092168092727661, 'eval_bleu': 5.6483, 'eval_gen_len': 17.5796, 'eval_runtime': 413.1979, 'eval_samples_per_second': 61.513, 'eval_steps_per_second': 3.846, 'epoch': 2.0}
{'train_runtime': 2570.8478, 'train_samples_per_second': 79.093, 'train_steps_per_second': 4.944, 'train_loss': 1.873049256859756, 'epoch': 2.0}


Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

To https://huggingface.co/cpb5867/my_awesome_opus_books_model
   f12f68e..f48bb62  main -> main

To https://huggingface.co/cpb5867/my_awesome_opus_books_model
   f48bb62..832f60c  main -> main



'https://huggingface.co/cpb5867/my_awesome_opus_books_model/commit/f48bb62a61d1684af8f1f86f238073d8c4d046f5'

In [9]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

translator = pipeline("translation", model="my_awesome_opus_books_model")
translator(text)

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

c:\Users\cpb5867\.conda\envs\TransConsf\lib\site-packages\transformers\pipelines\__init__.py:958: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


"Les légumes partagent des ressources avec des bactéries fixatrices d'azote."